In [1]:
import os
import json
with open("../ElaTBoT-DFT/ft_result/prompt4/test_result_repeat_in_bohrium_v100/generated_predictions.jsonl", 'r') as file:
    prediction_data = [json.loads(line) for line in file]
with open('../reproduce/data/prompt_type_4/ec_desc_test_dataset_with_mpid.json', 'r') as file:
    test_data = json.load(file)
with open("../reproduce/data/prompt_type_4/after_deleted_data.json", 'r') as file:
    filtered_data = json.load(file)

In [2]:
print(len(prediction_data))
print(len(test_data))
print(len(filtered_data))
print(prediction_data[0])
print(test_data[0])
print(filtered_data[0])

522
522
10520
{'label': '[[450.0, 195.0, 171.0, 0.0, 0.0, 0.0], [195.0, 470.0, 150.0, 0.0, 0.0, 0.0], [171.0, 150.0, 345.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 177.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 203.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 187.0]]', 'predict': '[[518.0, 170.0, 180.0, 0.0, 0.0, 0.0], [170.0, 520.0, 170.0, 0.0, 0.0, 0.0], [180.0, 170.0, 518.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 200.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 230.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 209.0]]'}
{'instruction': 'Given a material description, predict the elastic tensor of it directly and accurately with scientific logic. Answer without any other comments, descriptions, or explanations. The answer should be a 6x6 Python matrix. ', 'input': "The material FeB with a reduced composition of {'Fe': 1.0, 'B': 1.0} exhibits a density of 6.877 g/cm^3 and a density per atom of 8.047 g/cm^3. The information about the elements contained in the material is as follows. B has an electronegativity of 2.04, an ionization energy 

In [3]:
crystal_system_name = ['Cubic', 'Tetragonal', 'Hexagonal', 'Orthorhombic', 'Trigonal', 'Monoclinic', 'Triclinic']


In [8]:
# training set system distribution
from datasets import Dataset
# Check for data leakage
real_train_data = Dataset.from_json("/Users/siyuliu/Library/CloudStorage/OneDrive-TheUniversityofHongKong-Connect/Project/EC/source_code/reproduce/data/prompt_type_4/real_train_dataset_with_mpid.json")
    
train_mp_list = [item['material_id'] for item in real_train_data]

train_crystal_system_list = []
for i in range(len(train_mp_list)):
    for j in range(len(filtered_data)):
        if train_mp_list[i] == filtered_data[j]['material_id']:
            train_crystal_system_list.append(filtered_data[j]['crystal system'])
train_crystal_system_count = {}
for system in crystal_system_name:
    train_crystal_system_count[system] = train_crystal_system_list.count(system)
print("train_crystal_system_count:", train_crystal_system_count)


train_crystal_system_count: {'Cubic': 3792, 'Tetragonal': 1829, 'Hexagonal': 1320, 'Orthorhombic': 1248, 'Trigonal': 731, 'Monoclinic': 518, 'Triclinic': 60}


In [4]:
dataset = []
for i in range(len(test_data)):
    datapoint = {"material_id": test_data[i]['material_id'], "prediction": prediction_data[i]['predict'], "label": test_data[i]['output']} 
    for j in range(len(filtered_data)):
        if test_data[i]['material_id'] == filtered_data[j]['material_id']:
            datapoint['crystal_system'] = filtered_data[j]['crystal system']
    dataset.append(datapoint)
print(len(dataset))

522


In [5]:
import re
import ast

def get_pure_text(text):
    return re.match(r"\[\[.*?\]\]", text, re.DOTALL).group(0)

for i in range(len(dataset)):
    try:
        r = ast.literal_eval(dataset[i]['label'])
        p = eval(get_pure_text(dataset[i]['prediction']))
        dataset[i]['label'] = r
        dataset[i]['prediction'] = p
    except:
        print(i)
        print(dataset[i]['material_id'])
        continue

In [8]:
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
# Calculate MAE and R2 for each crystal system
results = {}
for system in crystal_system_name:
    system_data = [d for d in dataset if d.get('crystal_system') == system]
    if system_data:
        true_values = [np.mean(d['label']) for d in system_data]
        pred_values = [np.mean(d['prediction']) for d in system_data]
        
        mae = mean_absolute_error(true_values, pred_values)
        r2 = r2_score(true_values, pred_values)
        
        results[system] = {
            'count': len(system_data),
            'MAE': mae,
            'R2': r2
        }

# Print results
print("Performance metrics by crystal system:")
print("-" * 50)
print(f"{'Crystal System':<15} {'Count':>8} {'MAE':>10} {'R2':>10}")
print("-" * 50)
for system in crystal_system_name:
    if system in results:
        metrics = results[system]
        print(f"{system:<15} {metrics['count']:>8} {metrics['MAE']:>10.4f} {metrics['R2']:>10.4f}")

Performance metrics by crystal system:
--------------------------------------------------
Crystal System     Count        MAE         R2
--------------------------------------------------
Cubic                208     2.0291     0.9740
Tetragonal           101     2.3790     0.9555
Hexagonal             73     2.0483     0.9688
Orthorhombic          69     3.0048     0.9413
Trigonal              40     2.5931     0.9620
Monoclinic            28     2.9683     0.9335
Triclinic              3     2.0556    -3.0961


In [9]:
from pymatgen.analysis.elasticity import ElasticTensor
import numpy as np

def calculate_properties(stiffness_matrix: list) -> (float, float, float):
    elastic_constant = np.asarray(stiffness_matrix)
    elastic_tensor = ElasticTensor.from_voigt(elastic_constant)
    bulk_modulus = round(elastic_tensor.k_voigt, 3)
    return bulk_modulus

# calculate bulk modulus MAE and R2 for each crystal system
results_bulk_modulus = {}
for system in crystal_system_name:
    system_data = [d for d in dataset if d.get('crystal_system') == system]
    if system_data:
        true_values = [calculate_properties(d['label']) for d in system_data]
        pred_values = [calculate_properties(d['prediction']) for d in system_data]
        
        mae = mean_absolute_error(true_values, pred_values)
        r2 = r2_score(true_values, pred_values)
        
        results_bulk_modulus[system] = {
            'count': len(system_data),
            'MAE': mae,
            'R2': r2
        }

# Print results
print("Performance metrics by crystal system (Bulk Modulus):")
print("-" * 50)
print(f"{'Crystal System':<15} {'Count':>8} {'MAE':>10} {'R2':>10}")
print("-" * 50)
for system in crystal_system_name:
    if system in results_bulk_modulus:
        metrics = results_bulk_modulus[system]
        print(f"{system:<15} {metrics['count']:>8} {metrics['MAE']:>10.4f} {metrics['R2']:>10.4f}")

Performance metrics by crystal system (Bulk Modulus):
--------------------------------------------------
Crystal System     Count        MAE         R2
--------------------------------------------------
Cubic                208     7.0225     0.9720
Tetragonal           101     7.7359     0.9610
Hexagonal             73     7.0792     0.9570
Orthorhombic          69     9.3511     0.9452
Trigonal              40     8.7556     0.9627
Monoclinic            28     9.8373     0.9382
Triclinic              3     3.7037     0.2819
